<h1>Natural Language Processing</h1>

<h2>Stanford Sentiment Treebank</h2>

<p>A few years ago, Richard Socher put together a dataset for evaluating machine learning techniques to perform sentiment analysis. This dataset is interesting in that it includes a full parse tree for each sentence as well as a label representing how positive the sentiment is for that phrase for each branch in the parse tree.</p>

<p>Socher's original version is spread across several files and requires some overly complicated parsing, so I just downloaded Kaggle's version of this dataset, which is a pre-parsed-and-merged version distributed as a single tsv file.</p>

<p>If you'd like to play with this dataset, you can download it <a href=http://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data>here</a></p>

<p>To run this notebook, you will need:</p>
<p>numpy</p>
<p>scipy</p>
<p>pandas</p>
<p>matplotlib</p>
<p>scikit-learn</p>
<p>and, of course, ipython and the ipython notebook</p>

<br>
<p>To get started, let's load some common libraries for loading, transforming, and exploring text data...</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline
import os

<p>Let's read in the file we downloaded from Kaggle. After I unzipped the file, I renamed it from "train.tsv" to "movie_reviews_train.tsv" to avoid name clashes with other datasets. It's stored under "data" in my home folder.</p>

In [ ]:
phrases = pd.read_csv(os.path.join(os.environ['HOME'], 'data', 'movie_reviews_train.tsv'), sep='\t')

<p>To keep things simple, let's ignore the parse tree information. We'll only look at complete sentences and their corresponding labels.</p>

In [ ]:
phrases['word_count'] = phrases.Phrase.str.split().apply(len)
phrases = phrases.set_index(['SentenceId', 'word_count']).sort_index()
sentences = phrases.groupby(level=[0]).tail(1).reset_index()
phrases = phrases.reset_index()

<p>Socher's original paper that presented this dataset considers a 5-way classification problem with the following interpretation of the labels:</p>

<h5>Sentiment:</h5>
<p>0 -> Extremely negative</p>
<p>1 -> Slightly negative</p>
<p>2 -> Not negative or positive</p>
<p>3 -> Slightly positive</p>
<p>4 -> Extremely positive</p>

</br>

<p>Again, in the interest of simplicity, let's make this a binary classification problem. Socher did this in the original paper by assigning a "positive" label to label values of 3 and 4, and a "negative" label to label values of 0 and 1. In the binary framework, labels of 2 are excluded.</p>

In [ ]:
sentences.loc[sentences.Sentiment <= 1, 'binary_sentiment'] = 'negative'
sentences.loc[sentences.Sentiment >= 3, 'binary_sentiment'] = 'positive'
dataset = sentences.loc[sentences.binary_sentiment.notnull(), :]

<p>Always good practice to check the shape of your dataframe after filtering...</p>

In [ ]:
dataset.shape

<p>Here's what the dataframe looks like...</p>

In [ ]:
dataset.head(10)

<p>In the interest of competition, Kaggle doesn't make the labels for the test set available on this dataset. We could get the original dataset from Socher's website, but as I mentioned before, this involves some parsing and joining operations. In the interest of keeping things simple, let's just split the training set into a smaller training set and a test set with a 70/30 split.</p>

In [ ]:
from sklearn import cross_validation as cv

train_idx, test_idx = iter(cv.ShuffleSplit(n=dataset.shape[0], n_iter=1, test_size=0.3, random_state=123, indices=False)).next()
dataset.loc[train_idx, 'model_set'] = 'train'
dataset.loc[test_idx, 'model_set'] = 'test'
assert dataset.model_set.isnull().sum() == 0
train = dataset.loc[dataset.model_set=='train', :]
test = dataset.loc[dataset.model_set=='test', :]
print train.shape

<p>Let's get some summary stats!</p>

In [ ]:
print train.binary_sentiment.value_counts()
print ""
print "Fraction positive instances:\t{0}".format((train.binary_sentiment=='positive').mean())

<p>Let's see if there's any low-hanging fruit in the word counts. I wouldn't expect the number of words in a sentence to be predictive of sentiment, but it only takes a minute to check...</p>

In [ ]:
bins = np.linspace(0, 60, 20)
pl.figure(figsize=(16, 12))
pl.hist(train.loc[train.binary_sentiment=='positive', 'word_count'].values, histtype='stepfilled', color='b', alpha=0.5, normed=True, bins=bins)
pl.hist(train.loc[train.binary_sentiment=='negative', 'word_count'].values, histtype='stepfilled', color='r', alpha=0.5, normed=True, bins=bins)
pl.title('Word Counts -- positive and negative sentiment sentences');

<p>Hmmm... There's not a lot of signal in this variable. Maybe letters-per-word will look better?</p>

In [ ]:
bins = np.linspace(0, 10, 20)
train['letters_per_word'] = train.apply(lambda x: np.sum([len(word) for word in x.Phrase.split()]) / float(x.word_count), axis=1)
test['letters_per_word'] = test.apply(lambda x: np.sum([len(word) for word in x.Phrase.split()]) / float(x.word_count), axis=1)
pl.figure(figsize=(16, 12))
pl.hist(train.loc[train.binary_sentiment=='positive', 'letters_per_word'].values, histtype='stepfilled', color='b', alpha=0.5, normed=True, bins=bins)
pl.hist(train.loc[train.binary_sentiment=='negative', 'letters_per_word'].values, histtype='stepfilled', color='r', alpha=0.5, normed=True, bins=bins)
pl.title('Letters Per Word -- positive and negative sentiment sentences');

<p>Slightly better, but still not seeing a ton of separation. Let's see if some machine learning can help us here...</p>

<h2>Machine Learning with Text</h2>

<p>The classic procedure for building discriminative models from text is to create a gigantic, sparse matrix where each column represents a word in your dictionary (i.e. a word that appears somewhere in your training set) and each row represents a document (in this case, a sentence). Each cell can represent:</p>
<p>a. A binary indicator of whether or not that word is present in the sentence</p>
<p>b. A count of the number of times the word appears in that sentence, or</p>
<p>c. either "a." or "b." multiplied by some weighting scheme</p>
<br>
<p>We'll use tf/idf weighting, which is a popular choice for general natural language processing tasks. Usually binary indicators work better than word counts for sentiment analysis, so we'll use those for this example.</p>

In [ ]:
from scipy import sparse
from sklearn.feature_extraction import text

vec = text.TfidfVectorizer(binary=True,
                           lowercase=False) # lowercasing can reduce the column count, but we should see
                                            #  how many columns we end up with first
                           
X = vec.fit_transform(train.Phrase.values)
print X.shape

<p>It is not unusual to end up with more columns than rows in the resulting matrix, so any machine learning algorithm that uses this matrix must:</p>
<p>1. Scale to work with huge numbers of columns</p>
<p>2. Not produce degenerate solutions when the number of columns >= the number of rows, and</p>
<p>3. Be sufficiently regularized</p>
<p>We'll assume that words that appear only once or twice in our training set are more likely to lead to overfitting than to a better model and leave them out. We'll also lowercase each word in an effort to reduce the number of columns, which should help with overfitting as well. Let's see what the resulting shape of this matrix is and make a decision on the best algorithm to use to build a model.</p>

In [ ]:
vec = text.TfidfVectorizer(binary=True,
                           lowercase=True,
                           min_df = 3, # words must appear at least 3 times to be included in the matrix
                           norm=u'l2', # taking the L2 norm of each row after weighting is a trick that usually helps
                           use_idf=True, # this applies the weighting, which also usually helps
                           smooth_idf=True) # matters when you're considering low word counts, which we are here
X = vec.fit_transform(train.Phrase.values)
print X.shape

<p>Now we have 1/3 the number of columns as before, and we have fewer columns than rows. Despite the improvement, we still have a lot of columns, so we still probably want to start with something that is both sufficiently regularized and scales well to huge numbers of columns. Penalized logistic regression is a popular approach. L2 regularization is generally an easier and faster optimization than L1, so let's start with that.</p>
<p>We'll do a grid search for our regularization hyperparameter. Since accuracy is our metric, we can use lots of folds and still get a good estimate on out-of-sample performance while keeping our training set size big. We also won't worry about stratifying our folds because class imbalance is almost non-existant on this dataset</p>

In [ ]:
from sklearn import grid_search as gs
from sklearn import pipeline
from sklearn import metrics
from sklearn import linear_model as lm

param_grid = [{'model__C': 2.**np.linspace(-15, 15, 20)}]
folds = cv.KFold(n=train.shape[0],
                 n_folds=5,
                 shuffle=True,
                 indices=False,
                 random_state=1234)
model = lm.LogisticRegression(penalty='l2',
                              dual=False, #this is usually faster when the column count is higher than the row count
                              random_state=567)
pipe = pipeline.Pipeline([('vec', vec), ('model', model)])
searcher = gs.GridSearchCV(pipe, param_grid,
                           scoring='accuracy',
                           iid=True, # iid is never a great assumption, but it's not a horrible one here
                           cv=folds,
                           refit=False,
                           n_jobs=-1) # parallelize the grid search across all cores
searcher.fit(train.Phrase.values, train.binary_sentiment.values)
print "Best C:\t\t{0}".format(np.log2(searcher.best_params_['model__C']))
print "Best Score:\t{0}".format(searcher.best_score_)

pipe.named_steps['model'].C = searcher.best_params_['model__C']
for fold, (train_idx, test_idx) in enumerate(folds):
    pipe.fit(train.loc[train_idx, 'Phrase'].values, train.loc[train_idx, 'binary_sentiment'].values)
    train.loc[test_idx, 'fold'] = fold
    train.loc[test_idx, 'unigram_score'] = pipe.predict_proba(train.loc[test_idx, 'Phrase'].values)[:, 1]
pipe.fit(train.Phrase.values, train.binary_sentiment.values)
test['unigram_score'] = pipe.predict_proba(test.Phrase.values)[:, 1]
bins = np.linspace(0., 1., 20)
pl.figure(figsize=(16, 12))
pl.hist(train.loc[train.binary_sentiment=='positive', 'unigram_score'].values, histtype='stepfilled', color='b', alpha=0.5, normed=True, bins=bins)
pl.hist(train.loc[train.binary_sentiment=='negative', 'unigram_score'].values, histtype='stepfilled', color='r', alpha=0.5, normed=True, bins=bins)
pl.title('Unigram Score -- positive and negative sentiment sentences');

<p>Pretty substantial improvement, right? To be fair, the "C" hyperparameter was chosen based on the one that produced the best accuracy score over these cross validation folds. There is a small amount of data snooping present in the score, but since only one hyperparameter was chosen, I think it's safe to say that these scores are fairly representative of what we can expect to see on out-of-sample data.</p>

<h3>N-Grams</h3>
<p>One problem with the bag-of-words model is that we lose all the context for the words. As an example, consider the following sentences:</p>

<h5>Jack loves Jill.</h5>
<h5>Jill loves Jack.</h5>

<p>These sentences produce identical bag-of-words vector representations despite having very different meanings. One way that we might restore some context into the vector representation is to take each pair of words in addition to each single word. In the example above, we'd consider columns ['jack', 'loves', 'jill'] for each sentence, but the first sentence would have additional columns ['jack loves', 'loves jill'] and the second sentence would have additional columns ['jill loves', 'loves jack']. This is controlled with the "ngram_range" hyperparameter in the tf/idf vectorizer. Using bigrams, trigrams, etc... usually yields improvements in natural language processing tasks, although you should expect diminishing returns and overfitting at some point. Let's see if we get any improvement in our model's performance...</p>

In [ ]:
vec = text.TfidfVectorizer(binary=True,
                           lowercase=True,
                           ngram_range=(1, 2),
                           min_df = 3, # words must appear at least 3 times to be included in the matrix
                           norm=u'l2', # taking the L2 norm of each row after weighting is a trick that usually helps
                           use_idf=True, # this applies the weighting, which also usually helps
                           smooth_idf=True)

param_grid = [{'model__C': 2.**np.linspace(-15, 15, 20)}]
folds = cv.KFold(n=train.shape[0],
                 n_folds=5,
                 shuffle=True,
                 indices=False,
                 random_state=1234)
model = lm.LogisticRegression(penalty='l2',
                              dual=True, #this is usually faster when the column count is higher than the row count
                              random_state=567)
pipe = pipeline.Pipeline([('vec', vec), ('model', model)])
searcher = gs.GridSearchCV(pipe, param_grid,
                           scoring='accuracy',
                           iid=True, # iid is never a great assumption, but it's not a horrible one here
                           cv=folds,
                           refit=False,
                           n_jobs=-1) # parallelize the grid search across all cores
searcher.fit(train.Phrase.values, train.binary_sentiment.values)
print "Best C:\t\t{0}".format(np.log2(searcher.best_params_['model__C']))
print "Best Score:\t{0}".format(searcher.best_score_)

pipe.named_steps['model'].C = searcher.best_params_['model__C']
for fold, (train_idx, test_idx) in enumerate(folds):
    pipe.fit(train.loc[train_idx, 'Phrase'].values, train.loc[train_idx, 'binary_sentiment'].values)
    train.loc[test_idx, 'fold'] = fold
    train.loc[test_idx, 'bigram_score'] = pipe.predict_proba(train.loc[test_idx, 'Phrase'].values)[:, 1]
pipe.fit(train.Phrase.values, train.binary_sentiment.values)
test['bigram_score'] = pipe.predict_proba(test.Phrase.values)[:, 1]
bins = np.linspace(0., 1., 20)
pl.figure(figsize=(16, 12))
pl.hist(train.loc[train.binary_sentiment=='positive', 'bigram_score'].values, histtype='stepfilled', color='b', alpha=0.5, normed=True, bins=bins)
pl.hist(train.loc[train.binary_sentiment=='negative', 'bigram_score'].values, histtype='stepfilled', color='r', alpha=0.5, normed=True, bins=bins)
pl.title('Bigram Score -- positive and negative sentiment sentences');

<p>Well, we got a slight improvement, but not as big of one as we might have expected. Let's see what the shape of our document term matrix looks like with the additional columns...</p>

In [ ]:
bigramX = vec.fit_transform(train.Phrase.values)
bigramX.shape

<p>Looks like we have almost twice as many columns as before...</p>

In [ ]:
print X.data.shape[0] / float(np.prod(X.shape))
print bigramX.data.shape[0] / float(np.prod(bigramX.shape))

<p>And there lies the problem -- these additional bigram columns aren't as dense (they don't have as many non-zero components) as the unigram columns. This task is particularly challenging in that each document contains a single sentence, so sparsity is a problem even when only considering only unigrams. Looking at the above, even the unigram matrix only has 0.35% non-zero components. It's amazing that the algorithm was able to model this at all!</p>

<h4>For another discussion...</h4>
<p>Let's talk about how more modern techniques that attempt to solve the sparsity problem by rethinking the bag-of-words approach.</p>

<h3>Ensembling</h3>
<p>In the meantime, let's ensemble our two models. There are typically two approaches:</p>
<p>1. simple average</p>
<p>2. an ensemble model that combines the scores of the submodels</p>
<br>
<p>Since the simple average is simpler than the model, let's try that first:</p>

In [ ]:
print "Simple average accuracy: {0}".format(metrics.accuracy_score(train.binary_sentiment.values, np.where(train[['unigram_score', 'bigram_score']].mean(axis=1) > 0.5, 'positive', 'negative')))

<p>Not bad. We got a small improvement. Let's see if we do better with an ensemble model (a.k.a. a "stack").</p>

In [ ]:
param_grid = [{'C': 2.**np.linspace(-15, 15, 20)}]
folds = cv.KFold(n=train.shape[0],
                 n_folds=5,
                 shuffle=True,
                 indices=False,
                 random_state=1234)
model = lm.LogisticRegression(penalty='l2',
                              dual=False, #this is usually faster when the column count is higher than the row count
                              random_state=567)
searcher = gs.GridSearchCV(model, param_grid,
                           scoring='accuracy',
                           iid=True, # iid is never a great assumption, but it's not a horrible one here
                           cv=folds,
                           refit=True,
                           n_jobs=-1) # parallelize the grid search across all cores
searcher.fit(train[['unigram_score', 'bigram_score']].values, train.binary_sentiment.values)
print "Best C:\t\t{0}".format(np.log2(searcher.best_params_['C']))
print "Best Ensemble Score:\t{0}".format(searcher.best_score_)

<p>Even better! We'll use the stack to score our holdout set and see how well we expect to do on evaluating sentiment for sentences from movie reviews.</p>

In [ ]:
test['ensemble_score'] = searcher.predict_proba(test[['unigram_score', 'bigram_score']].values)[:, 1]
print "Ensemble Accuracy on Holdout Set: {0}".format(metrics.accuracy_score(test.binary_sentiment.values,
                                                                            searcher.predict(test[['unigram_score', 'bigram_score']].values)))
bins = np.linspace(0., 1., 20)
pl.figure(figsize=(16, 12))
pl.hist(test.loc[test.binary_sentiment=='positive', 'ensemble_score'].values, histtype='stepfilled', color='b', alpha=0.5, normed=True, bins=bins)
pl.hist(test.loc[test.binary_sentiment=='negative', 'ensemble_score'].values, histtype='stepfilled', color='r', alpha=0.5, normed=True, bins=bins)
pl.title('Ensemble Scores on Holdout Set');